# KIPRIS 기반 반도체 장비 AI 특허 데이터셋 구축 (Target-Prior Art Pair)

이 노트북은 **"반도체 장비 분야의 AI 활용 특허"**를 검색하고, 해당 특허의 **"심사관 인용 선행기술(Prior Art)"** 정보를 수집하여 **(Target Patent, Ground Truth Prior Art)** 쌍으로 구성된 데이터셋을 구축합니다.

이 데이터셋은 향후 **Agentic AI 기반 선행기술 조사 방법론**의 성능(Recall/Precision)을 검증하는 실험 데이터로 활용됩니다.

### 주요 단계
1. **환경 설정**: KIPRIS API Key 로드
2. **검색 쿼리 정의**: 반도체 장비(H01L 등) + AI 키워드 조합
3. **특허 검색 (Target 수집)**: `getAdvancedSearch` (항목별 검색) 활용
4. **인용 문헌 수집 (Ground Truth 수집)**: `getBibliographyDetailInfoSearch` (서지상세정보) 활용
5. **데이터셋 구축 및 저장**: JSONL 포맷으로 저장 (`data/processed/`)

In [1]:
import os
import time
import json
import requests
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv
from tqdm.auto import tqdm

# 1. 환경 설정 및 API 키 로드
def find_repo_root(start: Path | None = None) -> Path:
    cur = (start or Path.cwd()).resolve()
    for p in (cur, *cur.parents):
        if (p / "pyproject.toml").exists() and (p / "data").exists():
            return p
    return cur

REPO_ROOT = find_repo_root()
load_dotenv(REPO_ROOT / "env")
load_dotenv(REPO_ROOT / ".env")

KIPRIS_API_KEY = os.getenv("KIPRIS_API_KEY", "")
if not KIPRIS_API_KEY:
    raise ValueError("KIPRIS_API_KEY가 설정되지 않았습니다. env 파일을 확인하세요.")

# KIPRIS API 기본 설정
BASE_URL = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice"

# 오퍼레이션 경로
# 1) 항목별 검색 (Target 특허 검색용)
PATH_ADVANCED_SEARCH = "getAdvancedSearch"
# 2) 서지상세정보 (인용 문헌 확인용)
PATH_BIBLIO_DETAIL = "getBibliographyDetailInfoSearch"

print(f"API Key Loaded: {KIPRIS_API_KEY[:4]}... (Length: {len(KIPRIS_API_KEY)})")
print(f"Repo Root: {REPO_ROOT}")

# =========================
# 0. 수집 설정 (여기서 목표 건수/속도/재개/한도 대응 설정)
# - 셀 6은 아래 설정값을 그대로 사용합니다.
# =========================

# 모드 스위치
# - paper: 논문용(품질 우선) / 심사관 인용(Y)만 GT
# - experiment: 실험용(규모 우선) / GT 플래그 누락도 허용(논문에서는 silver-standard로 명시 권장)
MODE = "experiment"  # "paper" | "experiment"

# 목표: "유효 데이터"(prior arts 보유) 목표 건수
TARGET_VALID = 1500

# 실행 파라미터 (1500건 수집 안정성 우선 추천값)
# - ROWS_PER_PAGE를 올리면 검색 페이지 수가 줄어듭니다.
# - MAX_WORKERS는 너무 크면 API 실패/속도저하가 늘 수 있어 보수적으로 시작 추천
ROWS_PER_PAGE = 50
MAX_WORKERS = 5
MAX_PAGES_PER_STRATEGY = 5000     # 각 검색 전략별 최대 페이지 (전략 확장 대비 넉넉히)
MAX_EMPTY_PAGES = 30              # 연속 빈 페이지 허용치
MAX_PARSE_ERRORS = 5              # 연속 파싱 실패(None) 허용치

# 체크포인트(중간 저장) + 재개(resume) 설정
RESUME_FROM_CHECKPOINT = True
CHECKPOINT_EVERY_N = 50  # 새로 수집된 entry가 N개 쌓일 때마다 체크포인트에 append 저장

# 호출 제한(초당/일일 한도) 대응 설정
# - 정확한 일일 한도는 키/상품에 따라 다르므로, 429/한도초과 응답이 오면 안전하게 중단 후 resume 합니다.
# - THREAD가 있어도 전체적으로 요청 속도를 완만하게 유지하도록 최소 간격을 둡니다.
MIN_REQUEST_INTERVAL_SEC = 0.35  # 요청 간 최소 간격(초). 한도/차단이 잦으면 0.5~1.0으로 증가
STOP_ON_QUOTA = True  # 한도초과/차단 징후 감지 시 즉시 중단(체크포인트 저장 후 종료)

# 출력/체크포인트 경로
OUT_DIR = REPO_ROOT / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DATASET_BASENAME = "kipris_semiconductor_ai_dataset"
FINAL_JSONL_PATH = OUT_DIR / f"{DATASET_BASENAME}.jsonl"
CHECKPOINT_JSONL_PATH = OUT_DIR / f"{DATASET_BASENAME}.checkpoint.jsonl"
STATE_JSON_PATH = OUT_DIR / f"{DATASET_BASENAME}.state.json"

print("\n[수집 설정]")
print(f"MODE={MODE} | TARGET_VALID={TARGET_VALID} | ROWS_PER_PAGE={ROWS_PER_PAGE} | MAX_WORKERS={MAX_WORKERS}")
print(f"MIN_REQUEST_INTERVAL_SEC={MIN_REQUEST_INTERVAL_SEC} | STOP_ON_QUOTA={STOP_ON_QUOTA}")
print(f"CHECKPOINT: {CHECKPOINT_JSONL_PATH}")
print(f"STATE: {STATE_JSON_PATH}")
print("- 새로 시작하려면: RESUME_FROM_CHECKPOINT=False 또는 checkpoint/state 파일 삭제")

API Key Loaded: aggT... (Length: 44)
Repo Root: /home/arkwith/Dev/paper_data

[수집 설정]
MODE=experiment | TARGET_VALID=1500 | ROWS_PER_PAGE=50 | MAX_WORKERS=5
MIN_REQUEST_INTERVAL_SEC=0.35 | STOP_ON_QUOTA=True
CHECKPOINT: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.checkpoint.jsonl
STATE: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.state.json
- 새로 시작하려면: RESUME_FROM_CHECKPOINT=False 또는 checkpoint/state 파일 삭제


In [2]:
import threading
import time
from typing import Any, Dict

import requests
import xmltodict


class KiprisQuotaExceeded(RuntimeError):
    """일/분당 호출 제한(또는 차단)으로 더 이상 진행하면 안 되는 상태"""


def _looks_like_quota_error(result_msg: str | None) -> bool:
    if not result_msg:
        return False
    m = str(result_msg).lower()
    # 한국어/영문 혼합 케이스를 모두 포괄 (보수적으로 감지)
    keywords = ["트래픽", "제한", "초과", "quota", "limit", "rate", "too many", "429", "denied"]
    return any(k in m for k in keywords)


_kipris_lock = threading.Lock()
_kipris_last_request_ts = 0.0


def _throttle_kipris_requests() -> None:
    global _kipris_last_request_ts
    # 셀 2에서 설정한 MIN_REQUEST_INTERVAL_SEC 사용 (없으면 기본값)
    min_interval = float(globals().get("MIN_REQUEST_INTERVAL_SEC", 0.0) or 0.0)
    if min_interval <= 0:
        return
    with _kipris_lock:
        now = time.monotonic()
        wait = _kipris_last_request_ts + min_interval - now
        if wait > 0:
            time.sleep(wait)
        _kipris_last_request_ts = time.monotonic()


def kipris_get(path: str, params: Dict[str, Any], max_retries: int = 3) -> Dict[str, Any]:
    """KIPRIS API GET wrapper with retry/throttle and quota detection."""
    # - MIN_REQUEST_INTERVAL_SEC로 전체 요청 속도를 완만하게 유지
    # - 429/한도초과 응답(또는 resultMsg)을 감지하면 KiprisQuotaExceeded 예외 발생
    url = f"{BASE_URL}/{path}"
    params = params.copy()
    params["ServiceKey"] = KIPRIS_API_KEY

    stop_on_quota = bool(globals().get("STOP_ON_QUOTA", True))

    for attempt in range(max_retries):
        try:
            _throttle_kipris_requests()
            resp = requests.get(url, params=params, timeout=30)

            # HTTP 레벨 한도/차단
            if resp.status_code in {401, 403, 429}:
                msg = f"HTTP {resp.status_code} from KIPRIS"
                if stop_on_quota:
                    raise KiprisQuotaExceeded(msg)
                resp.raise_for_status()

            resp.raise_for_status()

            parsed = xmltodict.parse(resp.text)

            # 응답 헤더 기반 한도/오류 감지 (공공데이터 스타일)
            response = parsed.get("response") or {}
            header = response.get("header") or {}
            result_code = header.get("resultCode") or header.get("resultcode")
            result_msg = header.get("resultMsg") or header.get("resultmsg")

            if result_code and str(result_code) not in {"00", "0", "OK", "SUCCESS"}:
                if stop_on_quota and _looks_like_quota_error(result_msg):
                    raise KiprisQuotaExceeded(f"KIPRIS quota/limit: {result_msg}")
                # 그 외 에러는 파싱 결과를 그대로 반환(상위에서 처리)

            return parsed

        except KiprisQuotaExceeded:
            # 상위 루프에서 즉시 중단 + 체크포인트 flush 하도록 전달
            raise
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Request Failed: {url} | Error: {e}")
                return {}
            time.sleep(1 * (attempt + 1))
    return {}

In [3]:
# 2. 검색 쿼리 정의 (반도체 장비 + AI)
# IPC 코드: H01L (반도체 장치), G03F(포토리소그래피), C23C(증착), H01J(방전관/플라즈마) 등도 참고 가능
# 키워드: (반도체/웨이퍼/공정/장비) AND (인공지능+AI+머신러닝+딥러닝+신경망+강화학습)

# NOTE
# - KIPRIS 응답은 페이지/조건에 따라 item/items가 None으로 올 수 있어 견고한 파싱이 필요합니다.
# - 검색식이 너무 좁으면 결과 풀이 빨리 소진되어 TARGET_VALID에 도달하지 못할 수 있습니다.
# - 대규모 수집(예: 1500건)은 "전략을 점진적으로 넓히는" 방식이 안전합니다.

from dataclasses import dataclass
from typing import Optional


@dataclass(frozen=True)
class SearchStrategy:
    name: str
    astrtCont: Optional[str] = None
    inventionTitle: Optional[str] = None
    ipcNumber: Optional[str] = None


def _normalize_items(items: Any) -> List[Dict[str, Any]]:
    if items is None:
        return []
    if isinstance(items, dict):
        return [items]
    if isinstance(items, list):
        return [x for x in items if isinstance(x, dict)]
    return []


def search_target_patents(strategy: SearchStrategy, page: int = 1, rows: int = 20) -> Optional[List[Dict[str, Any]]]:
    """반도체 AI 관련 특허 검색

    Returns:
      - list[dict]: 정상 파싱된 결과(0개 가능)
      - None: 응답 포맷 문제/일시적 오류로 파싱 실패 (메인 루프에서 재시도/스킵 처리)
    """

    params: Dict[str, Any] = {
        "numOfRows": rows,
        "pageNo": page,
        "patent": "true",   # 특허만 (실용신안 제외)
        "utility": "false",
        "lastvalue": "",
    }

    # 조건은 None인 경우 파라미터 자체를 제외(빈 문자열로 넣는 것보다 안전)
    if strategy.astrtCont:
        params["astrtCont"] = strategy.astrtCont
    if strategy.inventionTitle:
        params["inventionTitle"] = strategy.inventionTitle
    if strategy.ipcNumber:
        params["ipcNumber"] = strategy.ipcNumber

    try:
        data = kipris_get(PATH_ADVANCED_SEARCH, params)
    except KiprisQuotaExceeded as e:
        # 상위 루프에서 처리할 수 있도록 다시 던짐
        raise

    try:
        response = data.get("response") or {}
        body = response.get("body") or {}
        items_container = body.get("items") or {}
        items = items_container.get("item")
        return _normalize_items(items)
    except Exception as e:
        print(f"Parsing Error on page {page} ({strategy.name}): {e}")
        return None


AI_KW_ABS = "(AI+인공지능+머신러닝+딥러닝+신경망+강화학습)"
SEMICON_KW = "(반도체+웨이퍼+공정+장비+노광+식각+증착+CVD+PVD+CMP+검사+측정+클리닝)"

# (논문용) 보수적: 도메인/품질 중시. 다만 유효 표본 수 확보를 위해 '검색만' 단계적으로 확장.
# Ground truth는 paper 모드에서 계속 examinerQuotationFlag == 'Y'만 사용합니다.
SEARCH_STRATEGIES_PAPER: List[SearchStrategy] = [
    SearchStrategy(
        name="paper_strict_H01L_title+abs",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle="반도체",
        ipcNumber="H01L",
    ),
    SearchStrategy(
        name="paper_relax_title_keep_H01L",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle=None,
        ipcNumber="H01L",
    ),
    SearchStrategy(
        name="paper_relax_ipc_keep_abs",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle=None,
        ipcNumber=None,
    ),
]

# (실험용) 공격적: 유효 N건 확보 목표, 단계적으로 검색 범위 확장
SEARCH_STRATEGIES_EXPERIMENT: List[SearchStrategy] = [
    # 1) 기존: 반도체 키워드 + AI 키워드 + H01L
    SearchStrategy(
        name="exp_strict_H01L_title+abs",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle="반도체",
        ipcNumber="H01L",
    ),
    SearchStrategy(
        name="exp_relax_title_keep_H01L",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle=None,
        ipcNumber="H01L",
    ),
    SearchStrategy(
        name="exp_relax_ipc_keep_abs",
        astrtCont="반도체*(인공지능+AI+학습+신경망)",
        inventionTitle=None,
        ipcNumber=None,
    ),
    SearchStrategy(
        name="exp_broader_abs",
        astrtCont="반도체*(AI+인공지능+머신러닝+딥러닝+신경망)",
        inventionTitle=None,
        ipcNumber=None,
    ),

    # 2) 추가: 반도체 용어 없이도, H01L 도메인 + AI 키워드로 확장 (결과 풀이 크게 증가하는 경우가 많음)
    SearchStrategy(
        name="exp_H01L_ai_abs_only",
        astrtCont=AI_KW_ABS,
        inventionTitle=None,
        ipcNumber="H01L",
    ),
    SearchStrategy(
        name="exp_H01L_ai_title_only",
        astrtCont=None,
        inventionTitle="AI+인공지능+머신러닝+딥러닝+신경망+강화학습",
        ipcNumber="H01L",
    ),

    # 3) 추가: 장비/공정 키워드(SEMICON_KW) + AI 키워드로 확장 (IPC 제약 완화)
    SearchStrategy(
        name="exp_equipment_kw_and_ai_abs",
        astrtCont=f"{SEMICON_KW}*{AI_KW_ABS}",
        inventionTitle=None,
        ipcNumber=None,
    ),

    # 4) 추가: 공정/장비 관련 IPC로 확장 (AI 키워드 유지)
    SearchStrategy(
        name="exp_G03F_ai_abs",
        astrtCont=AI_KW_ABS,
        inventionTitle=None,
        ipcNumber="G03F",
    ),
    SearchStrategy(
        name="exp_C23C_ai_abs",
        astrtCont=AI_KW_ABS,
        inventionTitle=None,
        ipcNumber="C23C",
    ),
    SearchStrategy(
        name="exp_H01J_ai_abs",
        astrtCont=AI_KW_ABS,
        inventionTitle=None,
        ipcNumber="H01J",
    ),
]

print("Search strategies ready:")
print("- paper:", [s.name for s in SEARCH_STRATEGIES_PAPER])
print("- experiment:", [s.name for s in SEARCH_STRATEGIES_EXPERIMENT])

Search strategies ready:
- paper: ['paper_strict_H01L_title+abs', 'paper_relax_title_keep_H01L', 'paper_relax_ipc_keep_abs']
- experiment: ['exp_strict_H01L_title+abs', 'exp_relax_title_keep_H01L', 'exp_relax_ipc_keep_abs', 'exp_broader_abs', 'exp_H01L_ai_abs_only', 'exp_H01L_ai_title_only', 'exp_equipment_kw_and_ai_abs', 'exp_G03F_ai_abs', 'exp_C23C_ai_abs', 'exp_H01J_ai_abs']


In [4]:
# 3. 인용 문헌(Prior Art) + 서지정보(Biblio) 수집 함수

def _to_list(x: Any) -> List[Dict[str, Any]]:
    if x is None:
        return []
    if isinstance(x, dict):
        return [x]
    if isinstance(x, list):
        return [i for i in x if isinstance(i, dict)]
    return []


def _first_or_none(x: Any) -> Optional[Dict[str, Any]]:
    xs = _to_list(x)
    return xs[0] if xs else None


def _dedupe_keep_order(values: List[str]) -> List[str]:
    out: List[str] = []
    seen: set[str] = set()
    for v in values:
        if not v:
            continue
        if v in seen:
            continue
        seen.add(v)
        out.append(v)
    return out


def get_biblio_detail_item(application_number: str) -> Dict[str, Any]:
    """KIPRIS 서지상세 item 원문(dict)을 반환"""
    params = {"applicationNumber": application_number}
    data = kipris_get(PATH_BIBLIO_DETAIL, params)
    return (data.get("response", {}) or {}).get("body", {}) or {}


def extract_biblio_metadata(body: Dict[str, Any]) -> Dict[str, Any]:
    """서지상세(body)에서 실험/논문에 유용한 메타데이터만 추출

    포함(가능한 범위):
      - IPC 분류 목록
      - 출원인/발명자(대표 1개 + 전체 리스트)
      - 등록 여부(등록/미등록/소멸 등) + 등록번호/등록일자
      - 패밀리/우선권 존재 여부와 요약
      - 법적상태(이벤트 수)
      - (추후 확장) CPC가 제공되면 동일 패턴으로 추가
    """

    item = (body.get("item") or {}) if isinstance(body, dict) else {}

    # Summary (등록여부 등)
    summary_arr = item.get("biblioSummaryInfoArray") or {}
    summary = _first_or_none(summary_arr.get("biblioSummaryInfo")) or {}

    def _clean_str(v: Any) -> Optional[str]:
        if v is None:
            return None
        s = str(v).strip()
        return s if s else None

    register_status = _clean_str(summary.get("registerStatus"))
    register_number = _clean_str(summary.get("registerNumber"))
    register_date = _clean_str(summary.get("registerDate"))
    publication_date = _clean_str(summary.get("publicationDate"))
    publication_number = _clean_str(summary.get("publicationNumber"))
    open_date = _clean_str(summary.get("openDate"))
    open_number = _clean_str(summary.get("openNumber"))
    final_disposal = _clean_str(summary.get("finalDisposal"))

    is_registered: Optional[bool] = None
    # KIPRIS 기준 registerStatus 예: 등록 / 소멸 등
    if register_number:
        is_registered = True
    elif register_status:
        if register_status in {"등록", "소멸", "존속", "무효"}:
            is_registered = True
        elif register_status in {"미등록", "거절", "취하", "포기"}:
            is_registered = False

    # IPC
    ipc_numbers: List[str] = []
    ipc_dates: List[str] = []
    ipc_arr = item.get("ipcInfoArray") or {}
    for x in _to_list(ipc_arr.get("ipcInfo")):
        ipc_numbers.append((x.get("ipcNumber") or "").strip())
        ipc_dates.append((x.get("ipcDate") or "").strip())

    ipc_numbers = _dedupe_keep_order(ipc_numbers)

    # Applicants / Inventors
    applicants: List[Dict[str, Any]] = []
    app_arr = item.get("applicantInfoArray") or {}
    for a in _to_list(app_arr.get("applicantInfo")):
        applicants.append({
            "name": a.get("name"),
            "eng_name": a.get("engName"),
            "country": a.get("country"),
            "code": a.get("code"),
        })

    inventors: List[Dict[str, Any]] = []
    inv_arr = item.get("inventorInfoArray") or {}
    for inv in _to_list(inv_arr.get("inventorInfo")):
        inventors.append({
            "name": inv.get("name"),
            "eng_name": inv.get("engName"),
            "country": inv.get("country"),
            "code": inv.get("code"),
        })

    # Priority / Family (구조가 케이스마다 달라서 '존재 + raw 요약' 중심)
    priority_arr = item.get("priorityInfoArray") or {}
    priority_list = _to_list(priority_arr.get("priorityInfo"))

    family_arr = item.get("familyInfoArray") or {}
    family_list = _to_list(family_arr.get("familyInfo"))

    # Legal status (요약)
    legal_arr = item.get("legalStatusInfoArray") or {}
    legal_list = _to_list(legal_arr.get("legalStatusInfo"))

    # CPC: 샘플에서는 없었지만, 응답에 존재하면 자동 추출(키명 변형 대비)
    cpc_numbers: List[str] = []
    for key in list(item.keys()):
        if "cpc" not in key.lower():
            continue
        maybe = item.get(key) or {}
        # 흔한 패턴: cpcInfoArray -> cpcInfo[{cpcNumber,...}]
        if isinstance(maybe, dict):
            for subk, subv in maybe.items():
                if "cpc" in subk.lower() and "info" in subk.lower():
                    for c in _to_list(subv):
                        num = c.get("cpcNumber") if isinstance(c, dict) else None
                        if num:
                            cpc_numbers.append(str(num).strip())

    cpc_numbers = _dedupe_keep_order(cpc_numbers)

    return {
        "classification": {
            "ipc": ipc_numbers,
            "cpc": cpc_numbers,
        },
        "registration": {
            "is_registered": is_registered,
            "register_status": register_status,
            "register_number": register_number,
            "register_date": register_date,
            "publication_date": publication_date,
            "publication_number": publication_number,
            "open_date": open_date,
            "open_number": open_number,
            "final_disposal": final_disposal,
        },
        "parties": {
            "applicants": applicants,
            "inventors": inventors,
        },
        "relations": {
            "priority_count": len(priority_list),
            "family_count": len(family_list),
        },
        "legal": {
            "events_count": len(legal_list),
        },
    }


def get_prior_art(
    application_number: str,
    *,
    policy: str = "paper",
    body: Optional[Dict[str, Any]] = None,
) -> List[str]:
    """출원번호로 상세 정보를 조회하여 인용 문헌 번호 리스트 반환

    - 대규모 수집에서는 API 호출 수를 줄이기 위해, 이미 가져온 서지상세 body를 주입할 수 있습니다.
    - body가 None이면 내부에서 KIPRIS를 호출합니다.

    policy:
      - "paper": 심사관 인용(examinerQuotationFlag == 'Y')만 GT로 사용 (품질 우선)
      - "experiment": 가능한 한 많이 수집 (플래그 누락/미정도 허용) (규모 우선)
    """
    if body is None:
        params = {"applicationNumber": application_number}
        data = kipris_get(PATH_BIBLIO_DETAIL, params)
        body = (data.get("response", {}) or {}).get("body", {}) or {}

    prior_arts: List[str] = []

    try:
        item = (body.get("item") or {}) if isinstance(body, dict) else {}
        p_array = item.get("priorArtDocumentsInfoArray", {}) or {}
        docs = _to_list(p_array.get("priorArtDocumentsInfo"))

        for p in docs:
            doc_num = p.get("documentsNumber")
            if not doc_num:
                continue

            flag = p.get("examinerQuotationFlag")

            if policy == "paper":
                if flag == "Y":
                    prior_arts.append(doc_num)
            else:
                # experiment: Y면 무조건 포함, 플래그가 없거나 다른 값이어도 포함(규모 우선)
                prior_arts.append(doc_num)

    except Exception:
        pass

    return _dedupe_keep_order([str(x).strip() for x in prior_arts])

In [5]:
# 4. 메인 수집 루프 (Target 검색 -> Prior Art 매핑) - 병렬 처리 + 체크포인트/재개(resume)
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

# 셀 2에서 설정한 값들을 사용
SEARCH_STRATEGIES = SEARCH_STRATEGIES_PAPER if MODE == "paper" else SEARCH_STRATEGIES_EXPERIMENT
PRIOR_ART_POLICY = "paper" if MODE == "paper" else "experiment"


def _load_checkpoint_jsonl(path: Path) -> tuple[list[Dict[str, Any]], set[str]]:
    """체크포인트 JSONL을 로드하여 dataset + seen_app_nums를 복원"""
    if not path.exists():
        return [], set()

    loaded: list[Dict[str, Any]] = []
    seen: set[str] = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                app = (obj.get("target_patent", {}) or {}).get("application_number")
                if not app:
                    continue
                if app in seen:
                    continue
                seen.add(app)
                loaded.append(obj)
            except Exception:
                continue
    return loaded, seen


def _write_state_json(*, dataset_count: int, seen_count: int, stop_reason: Optional[str] = None) -> None:
    state = {
        "updated_at": datetime.utcnow().isoformat() + "Z",
        "mode": MODE,
        "target_valid": TARGET_VALID,
        "rows_per_page": ROWS_PER_PAGE,
        "max_workers": MAX_WORKERS,
        "min_request_interval_sec": float(globals().get("MIN_REQUEST_INTERVAL_SEC", 0.0) or 0.0),
        "checkpoint_path": str(CHECKPOINT_JSONL_PATH),
        "dataset_count": dataset_count,
        "seen_app_nums_count": seen_count,
        "stop_reason": stop_reason,
    }
    with open(STATE_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=2)


# 0) 재개(resume): 기존 체크포인트가 있으면 먼저 로드
if RESUME_FROM_CHECKPOINT and CHECKPOINT_JSONL_PATH.exists():
    dataset, seen_app_nums = _load_checkpoint_jsonl(CHECKPOINT_JSONL_PATH)
    print(f"[RESUME] checkpoint 로드: {len(dataset)}건 (from {CHECKPOINT_JSONL_PATH})")
else:
    dataset = []
    seen_app_nums: set[str] = set()
    if RESUME_FROM_CHECKPOINT:
        print(f"[RESUME] checkpoint 없음. 새로 시작: {CHECKPOINT_JSONL_PATH}")
    else:
        print("[RESUME] 비활성화. 새로 시작")

checkpoint_written_app_nums: set[str] = set(seen_app_nums)
pending_checkpoint: list[Dict[str, Any]] = []

stop_early = False
stop_reason: Optional[str] = None


def _flush_checkpoint(force: bool = False) -> None:
    if not pending_checkpoint and not force:
        return
    if pending_checkpoint:
        CHECKPOINT_JSONL_PATH.parent.mkdir(parents=True, exist_ok=True)
        with open(CHECKPOINT_JSONL_PATH, "a", encoding="utf-8") as f:
            for entry in pending_checkpoint:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        pending_checkpoint.clear()
    _write_state_json(
        dataset_count=len(checkpoint_written_app_nums),
        seen_count=len(seen_app_nums),
        stop_reason=stop_reason,
    )


def process_patent(t: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """단일 특허 처리 함수 (병렬 실행용)

    대규모 수집 안정성을 위해, 서지상세(getBibliographyDetailInfoSearch)는 1회 호출만 하고
    그 body에서 prior art + biblio를 함께 추출합니다.
    """
    app_num = t.get("applicationNumber")
    if not app_num:
        return None

    # 1) 서지상세(1회 호출)
    body = get_biblio_detail_item(app_num)
    if not body:
        return None

    # 2) Prior arts (ground truth) - 이미 가져온 body 재사용
    prior_arts = get_prior_art(app_num, policy=PRIOR_ART_POLICY, body=body)
    if not prior_arts:
        return None

    # 3) Biblio detail metadata (IPC/CPC + parties + relations + legal summary)
    biblio_meta = extract_biblio_metadata(body)

    return {
        "target_patent": {
            "application_number": app_num,
            "title": t.get("inventionTitle"),
            "abstract": t.get("astrtCont"),
            "ipc": t.get("ipcNumber"),
            "applicant": t.get("applicantName"),
            "date": t.get("applicationDate"),
            "biblio": biblio_meta,
        },
        "ground_truth_prior_arts": prior_arts,
        "meta": {
            "source": "KIPRIS",
            "query_type": "semiconductor_ai",
            "mode": MODE,
            "search_policy": PRIOR_ART_POLICY,
        },
    }


print("=== 수집 시작 (병렬 처리 + 체크포인트) ===")
print(f"MODE={MODE} | TARGET_VALID={TARGET_VALID} | ROWS_PER_PAGE={ROWS_PER_PAGE} | MAX_WORKERS={MAX_WORKERS}")
print(f"FINAL: {FINAL_JSONL_PATH}")
print(f"CHECKPOINT: {CHECKPOINT_JSONL_PATH}")
print("SEARCH_STRATEGIES:")
for s in SEARCH_STRATEGIES:
    print("-", s.name)

if len(dataset) >= TARGET_VALID:
    print(f"[INFO] 이미 목표치를 충족했습니다: {len(dataset)}/{TARGET_VALID}")

# 통계
raw_returned_total = 0  # API가 반환한 원본 item 개수(중복 포함)
processed_unique_total = 0  # 중복 제거 후 실제 처리 시도한 건수

try:
    for strategy in SEARCH_STRATEGIES:
        if len(dataset) >= TARGET_VALID or stop_early:
            break

        print(f"\n--- Strategy: {strategy.name} ---")

        consecutive_empty_pages = 0
        consecutive_parse_errors = 0

        for page in tqdm(range(1, MAX_PAGES_PER_STRATEGY + 1), desc=f"Searching ({strategy.name})"):
            if len(dataset) >= TARGET_VALID or stop_early:
                break

            try:
                targets = search_target_patents(strategy, page=page, rows=ROWS_PER_PAGE)
            except KiprisQuotaExceeded as e:
                stop_early = True
                stop_reason = f"KIPRIS quota/rate limit detected during search: {e}"
                print(f"\n[STOP] {stop_reason}")
                _flush_checkpoint(force=True)
                break

            if targets is None:
                consecutive_parse_errors += 1
                print(f"Parsing failed on page {page}. (consecutive={consecutive_parse_errors}/{MAX_PARSE_ERRORS})")
                if consecutive_parse_errors >= MAX_PARSE_ERRORS:
                    print("Too many consecutive parsing failures for this strategy. Switching strategy.")
                    break
                continue

            consecutive_parse_errors = 0

            if not targets:
                consecutive_empty_pages += 1
                if consecutive_empty_pages >= MAX_EMPTY_PAGES:
                    print("No more results (consecutive empty pages). Switching strategy.")
                    break
                continue

            consecutive_empty_pages = 0
            raw_returned_total += len(targets)

            new_targets: List[Dict[str, Any]] = []
            for t in targets:
                app_num = t.get("applicationNumber")
                if not app_num:
                    continue
                if app_num in seen_app_nums:
                    continue
                seen_app_nums.add(app_num)
                new_targets.append(t)

            if not new_targets:
                continue

            processed_unique_total += len(new_targets)

            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                futures = [executor.submit(process_patent, t) for t in new_targets]

                for future in tqdm(as_completed(futures), total=len(futures), desc=f"Processing Page {page}", leave=False):
                    try:
                        result = future.result()
                        if not result:
                            continue
                        app_num = result["target_patent"]["application_number"]
                        if app_num in checkpoint_written_app_nums:
                            continue

                        dataset.append(result)
                        checkpoint_written_app_nums.add(app_num)
                        pending_checkpoint.append(result)

                        if len(pending_checkpoint) >= CHECKPOINT_EVERY_N:
                            _flush_checkpoint()

                        if len(dataset) >= TARGET_VALID:
                            break
                    except KiprisQuotaExceeded as e:
                        stop_early = True
                        stop_reason = f"KIPRIS quota/rate limit detected during detail fetch: {e}"
                        print(f"\n[STOP] {stop_reason}")
                        _flush_checkpoint(force=True)
                        break
                    except Exception as e:
                        print(f"Error processing patent: {e}")

            # 페이지 단위로도 한 번 flush 해서 커널 종료에 대비
            _flush_checkpoint()

            if len(dataset) >= TARGET_VALID:
                print(f"Reached TARGET_VALID={TARGET_VALID}. Stopping early.")
                break
            if stop_early:
                break

except KeyboardInterrupt:
    stop_early = True
    stop_reason = "Interrupted by user (KeyboardInterrupt)"
    print(f"\n[STOP] {stop_reason}")
    _flush_checkpoint(force=True)

# 루프 종료 후 마지막 flush
_flush_checkpoint(force=True)

print("\n=== 수집 완료 ===")
print(f"API 반환 item 수(원본, 중복 포함): {raw_returned_total}건")
print(f"처리 시도 특허 수(중복 제거): {processed_unique_total}건")
print(f"유효 데이터셋(인용문헌 보유) 수: {len(dataset)}건")

print("저장 데이터 유효율: 100% (저장되는 entry는 prior_arts 보유만 저장)")

if processed_unique_total:
    print(f"유효/처리시도 비율: {len(dataset)/processed_unique_total*100:.1f}%")
if raw_returned_total:
    print(f"유효/API반환 비율: {len(dataset)/raw_returned_total*100:.1f}%")

if stop_reason:
    print("\n[중단 사유]")
    print(stop_reason)
    print("- 다음날 이어서 수집하려면: 셀 2 실행 후, 다시 셀 6 실행 (RESUME_FROM_CHECKPOINT=True)")

if (len(dataset) < TARGET_VALID) and (not stop_reason):
    print("\n[안내] TARGET_VALID에 도달하지 못했습니다.")
    print("- 해결: SEARCH_STRATEGIES_EXPERIMENT에 더 넓은 전략 추가, 또는 ROWS_PER_PAGE/페이지 상향")
    print("- 참고: paper 모드는 GT 품질을 위해 더 보수적으로 수집됩니다.")

[RESUME] checkpoint 로드: 261건 (from /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.checkpoint.jsonl)
=== 수집 시작 (병렬 처리 + 체크포인트) ===
MODE=experiment | TARGET_VALID=1500 | ROWS_PER_PAGE=50 | MAX_WORKERS=5
FINAL: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.jsonl
CHECKPOINT: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.checkpoint.jsonl
SEARCH_STRATEGIES:
- exp_strict_H01L_title+abs
- exp_relax_title_keep_H01L
- exp_relax_ipc_keep_abs
- exp_broader_abs
- exp_H01L_ai_abs_only
- exp_H01L_ai_title_only
- exp_equipment_kw_and_ai_abs
- exp_G03F_ai_abs
- exp_C23C_ai_abs
- exp_H01J_ai_abs

--- Strategy: exp_strict_H01L_title+abs ---


Searching (exp_strict_H01L_title+abs):   0%|          | 0/5000 [00:00<?, ?it/s]

Processing Page 1:   0%|          | 0/23 [00:00<?, ?it/s]

Processing Page 2:   0%|          | 0/19 [00:00<?, ?it/s]

Processing Page 3:   0%|          | 0/15 [00:00<?, ?it/s]

...

Processing Page 34:   0%|          | 0/48 [00:00<?, ?it/s]

Processing Page 35:   0%|          | 0/50 [00:00<?, ?it/s]

Processing Page 36:   0%|          | 0/49 [00:00<?, ?it/s]

Processing Page 37:   0%|          | 0/50 [00:00<?, ?it/s]

Reached TARGET_VALID=1500. Stopping early.

=== 수집 완료 ===
API 반환 item 수(원본, 중복 포함): 3291건
처리 시도 특허 수(중복 제거): 2094건
유효 데이터셋(인용문헌 보유) 수: 1500건
저장 데이터 유효율: 100% (저장되는 entry는 prior_arts 보유만 저장)
유효/처리시도 비율: 71.6%
유효/API반환 비율: 45.6%


In [6]:
# 5. 데이터 저장 및 확인

# 전체 레코드 수
print(f"전체 데이터셋 레코드 수: {len(dataset)}")
print(f"FINAL_JSONL_PATH: {FINAL_JSONL_PATH}")
print(f"CHECKPOINT_JSONL_PATH: {CHECKPOINT_JSONL_PATH}")

# DataFrame 변환 (분석용)
df = pd.DataFrame([
    {
        "app_num": d["target_patent"]["application_number"],
        "title": d["target_patent"]["title"],
        "is_registered": (d.get("target_patent", {}).get("biblio", {}) or {}).get("registration", {}).get("is_registered"),
        "register_status": (d.get("target_patent", {}).get("biblio", {}) or {}).get("registration", {}).get("register_status"),
        "register_number": (d.get("target_patent", {}).get("biblio", {}) or {}).get("registration", {}).get("register_number"),
        "register_date": (d.get("target_patent", {}).get("biblio", {}) or {}).get("registration", {}).get("register_date"),
        "prior_art_count": len(d["ground_truth_prior_arts"]),
        "prior_arts": d["ground_truth_prior_arts"],
    }
    for d in dataset
])

print(f"DataFrame 행 수: {len(df)}")

print("데이터 미리보기:")
display(df.head())

# JSONL 저장 (최종본)
with open(FINAL_JSONL_PATH, "w", encoding="utf-8") as f:
    for entry in dataset:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"데이터셋 저장 완료: {FINAL_JSONL_PATH}")

전체 데이터셋 레코드 수: 1500
FINAL_JSONL_PATH: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.jsonl
CHECKPOINT_JSONL_PATH: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.checkpoint.jsonl
DataFrame 행 수: 1500
데이터 미리보기:


,app_num,title,is_registered,register_status,register_number,register_date,prior_art_count,prior_arts
0,1020240135833,학습형 반도체 공정 배기 제어 장치 및 방법,True,등록,10-2881040-0000,2025.10.30,2,"[EP00875811 A3, JP2014194966 A]"
1,1020180084572,반도체 패키지,True,등록,10-2048319-0000,2019.11.19,3,"[JP2002261167 A, JP2013062309 A, KR10201701129..."
2,1020200095371,공압 실린더의 동작 영상정보를 이용한 반도체 제조공정 감시 시스템과 감시 방법,True,소멸,10-2348881-0000,2022.01.04,5,"[KR1019990075536 A, KR1020080055789 A, JP11094..."
3,1020160161315,머신 러닝 기반 반도체 제조 수율 예측 시스템 및 방법,True,등록,10-1917006-0000,2018.11.02,3,"[JP3310009 B2, KR1020150018681 A, JP2005339587 A]"
4,1020190020440,반도체 시편의 검사에 사용가능한 훈련 세트의 생성,True,등록,10-2422111-0000,2022.07.13,5,"[KR1020140136713 A, US20110110578 A1, US201400..."


데이터셋 저장 완료: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.jsonl
